# Writing and retrieving co-simulation setups

This notebook shows how to write and retrieve co-simulation setups from a database.

**Note**: In order to have the 3DCityDB populated with the data required for this example, run notebook [DBLayerBasics](./DBLayerBasics.ipynb) exactly once.

## Example: A simple co-simulation setup

In the following, a simple co-simulation setup called `sim` is defined with the help of the [IntegrCiTy co-simulation platform](https://github.com/IntegrCiTy/zerobnl) (see also the [minimal example](https://github.com/IntegrCiTy/zerobnl/blob/develop/examples/MinimalExample/MinimalExample.ipynb) of package *zerobnl*)

In [1]:
import zerobnl
import os.path

# Create simulation setup.
sim = zerobnl.CoSim()

# Add meta model.
sim.create_meta_model(
    meta_model = 'MetaBase',
    list_of_attrs_to_set = [ ( 'a', 'unit' ) ],
    list_of_attrs_to_get = [ ( 'b', 'unit' ) ]
)

# Add environment for instances of the meta model.
sim.create_environment(
    env = 'EnvBase',
    wrapper = os.path.join( 'cosim', 'wrapper_base.py' ),
    dockerfile = os.path.join( 'cosim', 'Dockerfile_base' )
)

# Add node based on meta model and environment.
sim.add_node(
    node = 'Base0',
    meta = 'MetaBase',
    env = 'EnvBase',
    init_values = { 'c': .5 },
    files = [ os.path.join( 'cosim', 'dummy_file.txt' ) ]
)

# Add another node based on meta model and environment.
sim.add_node(
    node = 'Base1',
    meta = 'MetaBase',
    env = 'EnvBase',
    init_values = { 'c': .25 }
)

# Define links between nodes.
sim.add_link( get_node = 'Base0', get_attr = 'b', set_node = 'Base1', set_attr = 'a' )
sim.add_link( get_node = 'Base1', get_attr = 'b', set_node = 'Base0', set_attr = 'a' )

# Define simulation groups and sequence.
sim.create_sequence( [ [ 'Base0' ], [ 'Base1' ] ] )

# Define simulation time steps.
sim.set_time_unit( 'seconds' )
sim.create_steps( [15] * 4 * 60 )

## Storing a co-simulation setup to the database

Package DBLayer implements the mapping of the [Simulation Package](https://github.com/gioagu/3dcitydb_simulation_pkg) scheme to IntegrCiTy’s concepts for representing such a co-simulation setup, providing basically a persistence layer for co-simulation setups.
For storing zerobnl co-simulation setups to the extended 3DCityDB, package DBLayer provides class `DBWriter`.

In [2]:
from dblayer.zerobnl import *

# Define connection parameters.
connect = PostgreSQLConnectionInfo(
    user = 'postgres',
    pwd = 'postgres',
    host = 'localhost',
    port = '5432',
    dbname = 'citydb'
    )

# Create writer and connect to database.
writer = DBWriter( connect )

For the purpose of this notebook, no other co-simulation setups should be stored on the 3DCityDB with . To this end, the next line erases all data regarding co-simulation setups from the database. **Only execute the next line if you really want to erase all data regarding co-simulation setups from your database!**

In [3]:
writer.cleanup_simpkg_schema()

Upon connecting to the database, a co-simulation setup can be assigned a name and written to the database with the help of a single command.
Setting parameters `write_meta_models` and `write_envs` to `False` indicates that a co-simulation setup already defining meta-model `MetaBase` and model `EnvBase` have been previously written to the same database (otherwise the corresponding parameters should be omitted or set to `True`).

In [4]:
# Write co-simulation setup to database.
writer.write_to_db(
    sim,
    'TestSim1',
    write_meta_models = True,
    write_envs = True
    )

del sim

## Retrieving and running co-simulation setups

For reading co-simulation setups from the extended 3DCityDB, package DBLayer provides class `DBReader`.
Upon connecting to the database, a co-simulation setup stored in the database can be referred to by name and retrieved with the help of a single command.
In the example below, which simply reads back the co-simulation setup written in the previous example, the resulting object `sim_read` would be identical to object `sim` from above.

In [5]:
# Create reader and connect to database.
reader = DBReader( connect )

# Read co-simulation setup from database.
sim_read = reader.read_from_db( 'TestSim1' )

This co-simulation setup is ready to be used:

In [6]:
sim_read.run()
sim_read.connect_to_results_db()
sim_read.get_list_of_available_results()

INFO :: Simulation finished in 0 min and 7 sec


,IN/OUT,Node,Attribute
0,OUT,Base1,b
1,X,Base0,y
2,X,Base1,y
3,OUT,Base0,b


## Associate co-simulation setups with 3DCityDB data

In the current IntegrCiTy toolchain, the association of co-simulation setups with 3DCityDB data happens first and foremost by using the available information to parametrize simulation models.
In the context of urban energy systems simulation, this comprises not only scalar model parameters (e.g., U-values for walls) but also time-series data as provided via the Energy ADE (e.g., electrical load profiles).

The rather trivial approach for such an association is to directly retrieve certain values and setting them as initial values when defining the co-simulation setup.
However, package DBLayer and the Simulation Package also allow to persistently store associations with attributes of 3DCityDB objects and generic parameters.
This is demonstrated in the following code snippet, which basically extends the co-simulation setup shown above by defining an instance of class `AssociateCityDBObject` and assigning it to parameter `c` of node `Base0`.

In [7]:
# Create access point.
access = DBAccess()

# Connect to database.
access.connect_to_citydb( connect )

# Query the database using the conditions defined above.
heatpumps = access.get_citydb_objects( 'HeatPump' )
heatpump_id = heatpumps[0].id

# Define table and attribute (table column) name for association.
table = 'citydb_view.nrg8_conv_system_heat_pump'
attribute = 'nom_effcy'

# Define association with the help of the object ID (table row).
associate_object = AssociateCityDBObject(
    table_name = table,
    object_id = heatpump_id,
    column_name = attribute
    )

# Link attribute "c" of "Base0" with the association.
sim_read.nodes.loc[ 'Base0' ].InitVal[ 'c' ] = associate_object

# Write the co-simulation setup to the database (storing the association).
writer.write_to_db(
    sim_read,
    'TestSim2',
    write_meta_models = False,
    write_envs = False
    )

When retrieving a co-simulation setup with the help of class `DBReader` (see above), the link to the associated object is automatically resolved and the associated value is used.

In [8]:
del sim_read

sim_read = reader.read_from_db( 'TestSim2' )

print( sim_read.nodes.loc[ 'Base0' ].InitVal[ 'c' ] )

1.2


Done.